In [5]:
import sqlite3
import pandas as pd

db = sqlite3.connect('AMS.db')
query = 'select * from Order_Table'
df = pd.read_sql(query, con = db)
df['Date']=pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
dfMonth = df[(df['Year']==2019) & (df['Month']==12)]
dfOrderPL = dfMonth.groupby(['Date','OrderID'])[['SettleAmt']].sum()
dfOrderPL.reset_index()

,Date,OrderID,SettleAmt
0,2019-12-02,1,491.6
1,2019-12-02,2,101.6
2,2019-12-03,1,1691.6
3,2019-12-05,1,-564.4
4,2019-12-05,2,345.6
5,2019-12-05,3,95.6
6,2019-12-05,4,918.4
7,2019-12-09,1,975.6
8,2019-12-09,2,-774.4
9,2019-12-09,3,-554.4


In [2]:
dfMonth.groupby(['Date'])[['SettleAmt']].sum()

,SettleAmt
Date,
2019-12-02,593.2
2019-12-03,1691.6
2019-12-05,795.2
2019-12-09,-353.2
2019-12-10,171.2


In [3]:
df.groupby(['Year','Month'])[['SettleAmt']].sum()

SettleAmt
Year Month           
2019 11        2037.6
     12        2898.0

In [8]:
import pandas as pd
import portfolio
import sqlite3

db = sqlite3.connect('AMS.db')

dt = '2019-10-13'
account = ('CMS','CITIC')
df_trans = portfolio.get_trans(dt, account)

query = "SELECT SymbolCode, SymbolName, AssetClass,Sector FROM Symbol_Table"
df_symbol = pd.read_sql(query, con=db)

df = pd.merge(df_trans, df_symbol, on='SymbolCode', how='left')

df_stock = df[~df['AssetClass'].isin(['CASH'])]
#df_stock[['SymbolCode','SymbolName']]
df_stock = df_stock[['SymbolCode','Qty']]
df_stock_group = df_stock.groupby(['SymbolCode']).sum()
df_stock_group

,Qty
SymbolCode,
000726.SZ,0.0
002294.SZ,1000.0
200625.SZ,11900.0
510500.SS,5000.0
512880.SS,58500.0
600030.SS,1000.0
600177.SS,2800.0
601088.SS,2600.0
601166.SS,2800.0


In [9]:
df_trans_cur = df_trans
df_group_cur = df_trans_cur[['CurSettle', 'SettleAmt']]
df_group_cur = df_group_cur.groupby(['CurSettle']).sum()
#df_group_cur['SymbolCode'] = df_group_cur['CurSettle']
df_group_cur.rename(columns={'SettleAmt':'Qty'}, inplace=True)
df_group_cur.index.name = 'SymbolCode'
df_group_cur

,Qty
SymbolCode,
CNY,85957.60
HKD,515.32


In [10]:
df_hold = df_stock_group.append(df_group_cur)
df_hold = df_hold[~df_hold['Qty'].isin([0])]
df_hold.reset_index(inplace=True)
df_hold = pd.merge(df_hold, df_symbol, on='SymbolCode', how='left')
df_hold

,SymbolCode,Qty,SymbolName,AssetClass,Sector
0,002294.SZ,1000.00,信立泰,EQUITY,医药
1,200625.SZ,11900.00,长安B,EQUITY,汽车
2,510500.SS,5000.00,500ETF,EQUITY,综合指数
3,512880.SS,58500.00,证券ETF,EQUITY,证券
4,600030.SS,1000.00,中信证券,EQUITY,证券
5,600177.SS,2800.00,雅戈尔,EQUITY,多元企业
6,601088.SS,2600.00,中国神华,EQUITY,煤炭
7,601166.SS,2800.00,兴业银行,EQUITY,银行
8,CNY,85957.60,人民币,CASH,
9,HKD,515.32,港币,CASH,
